In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/project_klue')  # eda_nlp 경로를 추가
%cd /content/drive/MyDrive/project_klue
%pwd

Mounted at /content/drive
/content/drive/MyDrive/project_klue


'/content/drive/MyDrive/project_klue'

In [2]:
%ls

app.ipynb  google_play.ipynb            klue_pytorch_baseline.py  __pycache__/  submission/
eda.py     kclue.pt                     model/                    reviews2.csv  Untitled0.ipynb
flask      klue_pytorch_baseline.ipynb  prediction.csv            reviews.csv   wordnet.pickle


In [3]:
import os
print(os.getcwd())


/content/drive/MyDrive/project_klue


In [4]:
import os
os.system("ngrok http 5000 &")

0

In [5]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.5 MB/s eta 0:00:00


In [6]:
!pip install flask flask-ngrok


In [7]:
!pip install pyngrok==7.2.1

In [8]:
!ngrok authtoken 2ojpdD4dXyAi9g4wkViJoTTgPUt_kDfTqK2sneASPLUdD6HU

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
!pip install nbconvert
!jupyter nbconvert --to script klue_pytorch_baseline.ipynb

[NbConvertApp] Converting notebook klue_pytorch_baseline.ipynb to script
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('metadata' was unexpected)

Failed validating 'additionalProperties' in stream:

On instance['cells'][41]['outputs'][0]:
{'metadata': {'tags': None},
 'name': 'stderr',
 'output_type': 'stream',
 'text': '100%|██████████| 2482/2482 [10:02<00:00,  4.12it/s]\n'}
[NbConvertApp] Writing 39704 bytes to klue_pytorch_baseline.py


In [10]:
from flask import Flask, request, render_template, jsonify, redirect, url_for
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google_play_scraper import Sort, reviews, app
from pyngrok import ngrok  # Import ngrok for tunneling
import os
import requests
from flask_ngrok import run_with_ngrok
# from google_play import start, reviews_to_dataframe, fetch_app_reviews, get_package_id
from klue_pytorch_baseline import *
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/project_klue


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [35]:
def get_package_id(app_link):
    try:
        # Google Play URL에서 package ID 추출
        package_id = app_link.split("id=")[1].split("&")[0]
        return package_id
    except IndexError:
        raise ValueError("올바른 Google Play 링크를 입력하세요.")

# 앱 리뷰 가져오기
def fetch_app_reviews(app_link, count=100):
    package_id = get_package_id(app_link)
    result, _ = reviews(
        package_id,
        lang='ko',   # 리뷰 언어 설정
        country='kr', # 국가 설정
        sort=Sort.NEWEST,
        count=count,  # 가져올 리뷰 수
    )
    return result
# 데이터프레임 변환 함수
def reviews_to_dataframe(reviews_data):
    data = {
        "id": [f"TEST_{str(i).zfill(4)}" for i in range(len(reviews_data))],
        "문장": [review['content'] for review in reviews_data]
    }
    df = pd.DataFrame(data)
    df.to_csv('reviews2.csv', index=False, encoding='utf-8-sig')
    return df

# 앱 리뷰 가져오기 실행
def start(app_link):
    # app_link = input("Google Play 앱 링크를 입력하세요: ").strip()
    try:
        reviews_data = fetch_app_reviews(app_link)
        df = reviews_to_dataframe(reviews_data)
        # print(df[:10])
        return df
    except Exception as e:
        print(f"에러 발생: {e}")
        return 0

In [12]:
!ngrok config check

Valid configuration file at /root/.config/ngrok/ngrok.yml


In [47]:
#  submission = pd.read_csv('/content/sample_submission.csv')

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)
# Load the model and tokenizer
MODEL_PATH = "/content/drive/MyDrive/project_klue/kclue.pt"  # Path to the pre-trained model directory
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else fallback to CPU
assert os.path.exists(MODEL_PATH), f"Model file not found at {MODEL_PATH}"

BASE_MODEL_NAME = "klue/roberta-small"  # Replace with the correct base model name

# Load tokenizer and model from the specified path
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
model = torch.load(MODEL_PATH)
# model.tokenizer.model_input_names.remove("token_type_ids")

model.to(DEVICE)



# @app.route('/scrape', methods=['POST'])
def scrape_reviews(app_link):
    # app_link = request.form['app_link']

    df = start(app_link)  # Get the app link from the form input
    print('app_link', df.head(), 'col', df.columns)
    # df.to_csv('reviews.csv', index=False, encoding="utf-8-sig")
    # predict_response = requests.post("http://localhost:5000/predict")
    # df.to_csv('reviews2.csv')



# @app.route('/predict', methods=['POST', 'GET'])
def predict_reviews():
    # 각 레이블 클래스 예시
    type_labels = ["사실형", "추론형", "대화형", "예측형"]
    polarity_labels = ["긍정", "부정", "중립"]
    tense_labels = ["과거", "현재", "미래"]
    clarity_labels = ["확실", "불확실"]

    # LabelEncoder 생성
    type_decoder = LabelEncoder()
    polarity_decoder = LabelEncoder()
    tense_decoder = LabelEncoder()
    clarity_decoder = LabelEncoder()

    # 각 디코더에 클래스 레이블 학습
    type_decoder.fit(type_labels)
    polarity_decoder.fit(polarity_labels)
    tense_decoder.fit(tense_labels)
    clarity_decoder.fit(clarity_labels)

    # 디코더 리스트
    label_decoders = [type_decoder, polarity_decoder, tense_decoder, clarity_decoder]

    # Load the reviews from the CSV file
    review_df = pd.read_csv('reviews2.csv')  # 리뷰 데이터가 저장된 CSV 파일
    print('col', review_df.columns)
    # dataset
    test_dataloader = DataLoader(SentenceTypeDataset(review_df, tokenizer), batch_size=32, shuffle=False)

    for batch in test_dataloader:
        print('batch', batch)  # 데이터가 올바르게 로드되는지 확인
        break

    # Perform predictions using the loaded model
    print('predict start!!!!!!!!!!!!!!!!!!!!!!')
    test_pred_type, test_pred_polarity, test_pred_tense, test_pred_certainty = get_type_predictions(model, test_dataloader)  # 예측값 생성
    print("sum:::::::::::::::::::::::::::::;;", sum(test_pred_type[0]))
    test_type = ['대화형' if i==0 else '사실형' if i==1 else '예측형' if i==2 else '추론형' for i in [np.argmax(p) for p in test_pred_type]]
    test_polarity = ['긍정' if i==0 else '미정' if i==1 else '부정' for i in [np.argmax(p) for p in test_pred_polarity]]
    test_tense = ['과거' if i==0 else '미래' if i==1 else '현재' for i in [np.argmax(p) for p in test_pred_tense]]
    test_certainty = ['불확실' if i==0 else '확실' for i in [np.argmax(p) for p in test_pred_certainty]]

    # Save the updated DataFrame to a new CSV file
    label_sum = []
    for i in range(len(test_type)):
        print('length :', len(review_df.iloc[i]['문장']))
        label_sum.append(f'{test_type[i]}-{test_polarity[i]}-{test_tense[i]}-{test_certainty[i]}')
    # os.makedirs('submission', exist_ok=True)

    ids = [f"TEST_{str(i+1).zfill(3)}" for i in range(len(label_sum))]
    results_df = pd.DataFrame({
    "ID": ids,
    "문장": review_df['문장'].tolist(),
    "label": label_sum
    })
    results_df.to_csv("prediction.csv", index=False, encoding='utf-8-sig')
    print(pd.read_csv("prediction.csv"))

app_link = input("Google Play 앱 링크를 입력하세요: ").strip()
scrape_reviews(app_link) # 크롤링
predict_reviews() # 예측 csv 생성

<ipython-input-47-facfcf354370>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(MODEL_PATH)


Google Play 앱 링크를 입력하세요: https://play.google.com/store/apps/details?id=com.ncsoft.lineagem19&pcampaignid=merch_published_cluster_promotion_battlestar_evergreen_dynasty_games&hl=ko
app_link           id                                                 문장
0  TEST_0000                            게임은 안해봤지만 NC라서 별 한개줌 ㅇㅇ
1  TEST_0001  윽 매일 전설 도전할때마다 실패네 ㅜ ㅜ 나보다 늦게 한 린m 시작한 분은 다 전설...
2  TEST_0002                                             넘재미있어요
3  TEST_0003  마을마다 졸라 멍때리고 있는거 정리해서 접속대기 안생기게 안되나? 동접 뻥티기 하고...
4  TEST_0004  접 대기자 기본이 300~400명가량 되던데 겜하다가 팅겨서 제접하려면 좀시간이 걸... col Index(['id', '문장'], dtype='object')
col Index(['id', '문장'], dtype='object')
batch [{'input_ids': tensor([[[    0,  4171,  2073,  ...,     1,     1,     1]],

        [[    0,  1496,  4709,  ...,     1,     1,     1]],

        [[    0,   749,  2070,  ...,     1,     1,     1]],

        ...,

        [[    0,  1041,  2205,  ...,     1,     1,     1]],

        [[    0,    46, 30630,  ...,     1,     1,     1]],

       

100%|██████████| 4/4 [00:00<00:00, 14.04it/s]


sum:::::::::::::::::::::::::::::;; 0.9999999757856131
length : 23
length : 129
length : 6
length : 113
length : 149
length : 3
length : 58
length : 125
length : 84
length : 84
length : 18
length : 55
length : 40
length : 355
length : 66
length : 16
length : 116
length : 64
length : 60
length : 14
length : 53
length : 19
length : 192
length : 2
length : 42
length : 65
length : 3
length : 7
length : 89
length : 37
length : 19
length : 2
length : 42
length : 34
length : 93
length : 20
length : 3
length : 7
length : 38
length : 36
length : 142
length : 6
length : 21
length : 6
length : 10
length : 45
length : 18
length : 79
length : 245
length : 71
length : 219
length : 68
length : 25
length : 21
length : 13
length : 8
length : 6
length : 41
length : 6
length : 3
length : 11
length : 47
length : 9
length : 197
length : 30
length : 1
length : 53
length : 8
length : 36
length : 36
length : 1
length : 65
length : 27
length : 87
length : 7
length : 12
length : 90
length : 5
length : 5
length :

In [48]:
app = Flask(__name__)


@app.route('/')
def home():
    return '''
    <!doctype html>
    <html>
    <head>
        <title>Polarity Selection</title>
    </head>
    <body>
        <h1>Select Polarity</h1>
        <form action="/filter" method="post">
            <button type="submit" name="polarity" value="긍정">긍정</button>
            <button type="submit" name="polarity" value="부정">부정</button>
            <button type="submit" name="polarity" value="미정">미정</button>
        </form>
    </body>
    </html>
    '''

@app.route('/filter', methods=['POST'])
def filter_reviews():
    try:
        polarity = request.form['polarity']

        # Load the predictions DataFrame
        df = pd.read_csv("prediction.csv")

        # Filter the DataFrame by the selected polarity
        filtered_df = df[df['label'].str.contains(polarity)]

        # Check if the filtered DataFrame is empty
        if filtered_df.empty:
            return f"<html><body><h1>'{polarity}' 문장이 없습니다!</h1><br><a href='/'>홈으로</a></body></html>"

        # Convert filtered DataFrame to HTML and return it as a response
        filtered_html = filtered_df[['ID', '문장', 'label']].to_html(index=False, escape=False)
        row_count = len(filtered_df)
        return f"<html><body><h1>{polarity} 문장</h1><h2>({row_count} 개)</h2><a href='/'>홈으로</a><br>{filtered_html}<br></body></html>"

    except Exception as e:
        return f"<html><body>Error: {str(e)}<br><a href='/'>홈으로</a></body></html>"


if __name__ == '__main__':
    # Set up ngrok tunnel
    public_url = ngrok.connect(5000)  # Expose Flask app on port 5000
    print(f"Ngrok Tunnel URL: {public_url}")

    # Run the Flask app
    app.run()


Ngrok Tunnel URL: NgrokTunnel: "https://d24e-34-87-82-196.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Dec/2024 13:00:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Dec/2024 13:00:42] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Dec/2024 13:00:42] "POST /filter HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Dec/2024 13:01:08] "POST /filter HTTP/1.1" 200 -
